In [1]:
import os

from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. Dataset preparation + train/val/test split

In [2]:
DATA_DIR = Path(os.getcwd()).parent / "data" / "train_data"

df = pd.read_csv(DATA_DIR / "ukraine_all_data.csv")
df.head()

,cell,city,is_positive,geometry,bench_count,cafe_count,pharmacy_count,waste_disposal_count,atm_count,post_office_count,...,wash_count,water_crane_count,farmland_count,forest_count,grass_count,allotments_count,meadow_count,cemetery_count,orchard_count,farmyard_count
0,881e7159e9fffff,Ярмолинці,1,POLYGON ((26.83393817535367 49.192988306649454...,0,0,1,0,0,1,...,0,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,881e6d3a5bfffff,Тростянець,1,POLYGON ((34.95934457201796 50.473494753294666...,11,1,4,0,3,2,...,0,0,1.0,NaN,34.0,2.0,NaN,NaN,NaN,NaN
2,881e6824dbfffff,Полтава,1,"POLYGON ((34.49924943744107 49.5775947405843, ...",10,1,2,3,1,1,...,0,0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN
3,881e6c32c3fffff,Ромни,1,"POLYGON ((33.49082533801754 50.7484969657831, ...",0,0,1,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,881e652d99fffff,Чернігів,1,POLYGON ((31.283317311257992 51.49889221417866...,0,0,0,0,0,0,...,0,0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7084 entries, 0 to 7083
Columns: 107 entries, cell to farmyard_count
dtypes: float64(18), int64(85), object(4)
memory usage: 5.8+ MB


In [4]:
sorted(df.columns.to_list())

['allotments_count',
 'apartments_count',
 'apteka_dobrego_dnya_count',
 'apteka_nyzkykh_zin_count',
 'apteka_optovykh_zin_count',
 'apteka_podorozhnyk_count',
 'atb_count',
 'atm_count',
 'bank_count',
 'bar_count',
 'bench_count',
 'bicycle_parking_count',
 'border_count',
 'buffer_stop_count',
 'bureau_de_change_count',
 'cafe_count',
 'car_wash_count',
 'cell',
 'cemetery_count',
 'city',
 'clinic_count',
 'community_centre_count',
 'construction_count',
 'crossing_count',
 'crossover_count',
 'dentist_count',
 'derail_count',
 'detached_count',
 'doctors_count',
 'drinking_water_count',
 'eko_market_count',
 'farmland_count',
 'farmyard_count',
 'fast_food_count',
 'fora_count',
 'forest_count',
 'fuel_count',
 'garage_count',
 'garages_count',
 'geometry',
 'grass_count',
 'halt_count',
 'house_count',
 'hump_yard_count',
 'industrial_count',
 'is_positive',
 'junction_count',
 'level_crossing;crossing_count',
 'level_crossing_count',
 'library_count',
 'meadow_count',
 'mileston

In [5]:
metadata_cols = ["cell", "city", "geometry", "wkt"]

df_metadata = df[metadata_cols]
df_data = df.drop(columns=metadata_cols)
target_col = "is_positive"

In [6]:
city_vc = df_metadata[df_data['is_positive'].astype(bool)]['city'].value_counts()
city_vc[city_vc > 20]

city
Київ             152
Дніпро            67
Харків            67
Кривий Ріг        52
Запоріжжя         42
Одеса             37
Полтава           36
Миколаїв          32
Львів             25
Кропивницький     23
Суми              23
Чернігів          23
Кам'янське        22
Name: count, dtype: int64

In [7]:
cols_with_nan = df_data.columns[df_data.isna().any()].to_list()

# fill cols with nan with 0 since they all are counts
df_data[cols_with_nan] = df_data[cols_with_nan].fillna(0)

In [8]:
test_cities = ['Львів', 'Харків', 'Одеса']

df_data_test = df_data[df_metadata['city'].isin(test_cities)]
df_data_train = df_data[~df_metadata['city'].isin(test_cities)]

X_train = df_data_train.drop(columns=[target_col])
y_train = df_data_train[target_col]

X_test = df_data_test.drop(columns=[target_col])
y_test = df_data_test[target_col]

In [31]:
X_test

,bench_count,cafe_count,pharmacy_count,waste_disposal_count,atm_count,post_office_count,bank_count,restaurant_count,waste_basket_count,fuel_count,...,wash_count,water_crane_count,farmland_count,forest_count,grass_count,allotments_count,meadow_count,cemetery_count,orchard_count,farmyard_count
351,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0
359,4,5,10,11,4,4,5,0,0,0,...,0,0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,1.0,13.0,1.0,0.0,0.0,0.0,0.0
401,0,1,5,0,1,2,2,0,0,0,...,0,0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
407,0,1,5,1,5,2,1,2,0,0,...,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6580,0,1,1,0,0,1,1,0,0,0,...,0,0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6581,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6582,0,0,0,0,0,0,0,0,0,0,...,0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6583,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"Train set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Train set size: 5244
Validation set size: 1312
Test set size: 528


In [32]:
print(f"Number of positive samples in train set: {y_train.sum()} | negative: {len(y_train) - y_train.sum()}")
print(f"Number of positive samples in validation set: {y_val.sum()} | negative: {len(y_val) - y_val.sum()}")
print(f"Number of positive samples in test set: {y_test.sum()} | negative: {len(y_test) - y_test.sum()}")

Number of positive samples in train set: 1255 | negative: 3989
Number of positive samples in validation set: 314 | negative: 998
Number of positive samples in test set: 129 | negative: 399


In [10]:
# check X_train types
X_train.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [12]:
data = {
    "X_train_scaled": X_train_scaled,
    "y_train": y_train,
    "X_val_scaled": X_val_scaled,
    "y_val": y_val,
    "X_test_scaled": X_test_scaled,
    "y_test": y_test,
}

# 2. Training

In [13]:
def compute_metrics(data, model, model_name, verbose=True):
    results = {}
    splits = ['train', 'val', 'test']

    for split in splits:
        X = data[f'X_{split}_scaled']
        y = data[f'y_{split}']
        
        y_pred = model.predict(X)
        accuracy = accuracy_score(y, y_pred)
        report = classification_report(y, y_pred, output_dict=True)
        f1_score = report['1']['f1-score']
        precision = report['1']['precision']
        recall = report['1']['recall']

        to_percantage = lambda x: np.round(x * 100, 3)

        results[split] = {
            'accuracy': to_percantage(accuracy),
            'f1_score': to_percantage(f1_score),
            'precision': to_percantage(precision),
            'recall': to_percantage(recall),
        }
    
        if verbose:
            print(f"{model_name} - {split} set metrics:")
            print(f"Accuracy: {accuracy:.4f}, F1 Score: {f1_score:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
            print(classification_report(y, y_pred))

    return results


## 2.1. Baseine Logistic regression

In [14]:
# checj which columns has NaN values
nan_cols = X_train.columns[X_train.isna().any()].tolist()
if nan_cols:
    print(f"Columns with NaN values: {nan_cols}")

In [15]:
print("Any NaN?", np.isnan(X_train_scaled).any())
print("Any ±inf?", np.isinf(X_train_scaled).any())
print("Feature ranges:", X_train_scaled.min(axis=0), X_train_scaled.max(axis=0))

Any NaN? False
Any ±inf? False
Feature ranges: [-0.14527252 -0.18696475 -0.21801917 -0.15707255 -0.18763595 -0.28459122
 -0.18749523 -0.15580738 -0.08258855 -0.22451987 -0.06224243 -0.18748882
 -0.15292978 -0.173111   -0.10676861 -0.12823332 -0.13959524 -0.16383576
 -0.13106569 -0.12644685 -0.19946854 -0.09300622 -0.11726633 -0.17021339
 -0.11773076 -0.0654196  -0.14688009 -0.14510222 -0.1123136  -0.06131132
 -0.19427445 -0.04572132 -0.07292416 -0.12368431 -0.08648482 -0.17192718
 -0.07884733 -0.11424363 -0.54424159 -0.29092205 -0.12367565 -0.36047254
 -0.48714007 -0.10400653 -0.46920825 -0.08751085 -0.24087503 -0.04539758
 -0.17536402 -0.09775393 -0.15029888 -0.12365742 -0.07343058 -0.11000028
 -0.11548891 -0.06058068 -0.15782168 -0.06938515 -0.04146321 -0.07095008
 -0.06319329 -0.02392509 -0.3328586  -0.01953289 -0.01381052 -0.02610583
 -0.03223822  0.         -0.01953289 -0.01381052 -0.01381052 -0.2502887
 -0.01381052 -0.01381052 -0.01381052  0.          0.          0.
  0.         

In [16]:
lr = LogisticRegression(penalty='l2', C=0.1, solver='lbfgs', max_iter=1000, random_state=42)

lr.fit(X_train_scaled, y_train)

/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:330: RuntimeWarning: divide by zero encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/linear_mo

,penalty,'l2'
,dual,False
,tol,0.0001
,C,0.1
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [17]:
lr_metrics = compute_metrics(
    data, lr, "Logistic Regression", verbose=True
)

Logistic Regression - train set metrics:
Accuracy: 0.9268, F1 Score: 0.8323, Precision: 0.9208, Recall: 0.7594
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      3989
           1       0.92      0.76      0.83      1255

    accuracy                           0.93      5244
   macro avg       0.92      0.87      0.89      5244
weighted avg       0.93      0.93      0.92      5244

Logistic Regression - val set metrics:
Accuracy: 0.9177, F1 Score: 0.8118, Precision: 0.8962, Recall: 0.7420
              precision    recall  f1-score   support

           0       0.92      0.97      0.95       998
           1       0.90      0.74      0.81       314

    accuracy                           0.92      1312
   macro avg       0.91      0.86      0.88      1312
weighted avg       0.92      0.92      0.91      1312

Logistic Regression - test set metrics:
Accuracy: 0.9072, F1 Score: 0.8350, Precision: 0.7381, Recall: 0.9612
              pr

/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: Runtim

In [18]:
print(f"Feature importances (coefficients):")
sorted_indices = np.argsort(lr.coef_[0])

# print the top 10 positive and negative coefficients
top_positive_indices = sorted_indices[-10:]
top_negative_indices = sorted_indices[:10]

print("Top 10 positive coefficients:")
for idx in top_positive_indices:
    print(f"{X_train.columns[idx]}: {lr.coef_[0][idx]:.4f}")

print("\nTop 10 negative coefficients:")
for idx in top_negative_indices:
    print(f"{X_train.columns[idx]}: {lr.coef_[0][idx]:.4f}")

Feature importances (coefficients):
Top 10 positive coefficients:
waste_basket_count: 0.5175
atb_count: 0.6002
community_centre_count: 0.6667
apteka_optovykh_zin_count: 0.6753
atm_count: 0.7341
retail_count: 0.9116
bank_count: 1.1002
pharmacy_count: 1.2099
post_office_count: 1.5512
apartments_count: 1.6885

Top 10 negative coefficients:
restaurant_count: -0.7854
clinic_count: -0.5847
stop_position_count: -0.5212
apteka_dobrego_dnya_count: -0.4465
novus_count: -0.4155
buffer_stop_count: -0.4024
doctors_count: -0.3862
fast_food_count: -0.3711
farmland_count: -0.3566
payment_terminal_count: -0.3226


## 2.1. Decision Tree + Random Forest

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Train a Decision Tree Classifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train_scaled, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [20]:
dt_metrics = compute_metrics(
    data, dt, "Decision Tree Classifier", verbose=True
)

Decision Tree Classifier - train set metrics:
Accuracy: 0.9992, F1 Score: 0.9984, Precision: 1.0000, Recall: 0.9968
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3989
           1       1.00      1.00      1.00      1255

    accuracy                           1.00      5244
   macro avg       1.00      1.00      1.00      5244
weighted avg       1.00      1.00      1.00      5244

Decision Tree Classifier - val set metrics:
Accuracy: 0.9070, F1 Score: 0.8076, Precision: 0.8000, Recall: 0.8153
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       998
           1       0.80      0.82      0.81       314

    accuracy                           0.91      1312
   macro avg       0.87      0.88      0.87      1312
weighted avg       0.91      0.91      0.91      1312

Decision Tree Classifier - test set metrics:
Accuracy: 0.8504, F1 Score: 0.7476, Precision: 0.6359, Recall: 0.9070
 

In [21]:
rf = RandomForestClassifier(random_state=42)

rf.fit(X_train_scaled, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [22]:
rf_metrics = compute_metrics(
    data, rf, "Random Forest Classifier", verbose=True
)

Random Forest Classifier - train set metrics:
Accuracy: 0.9992, F1 Score: 0.9984, Precision: 1.0000, Recall: 0.9968
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3989
           1       1.00      1.00      1.00      1255

    accuracy                           1.00      5244
   macro avg       1.00      1.00      1.00      5244
weighted avg       1.00      1.00      1.00      5244

Random Forest Classifier - val set metrics:
Accuracy: 0.9360, F1 Score: 0.8671, Precision: 0.8616, Recall: 0.8726
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       998
           1       0.86      0.87      0.87       314

    accuracy                           0.94      1312
   macro avg       0.91      0.91      0.91      1312
weighted avg       0.94      0.94      0.94      1312

Random Forest Classifier - test set metrics:
Accuracy: 0.8693, F1 Score: 0.7837, Precision: 0.6579, Recall: 0.9690
 

## pruning (DecisionTree only)

In [23]:
from sklearn.model_selection import GridSearchCV

dt_prunned = DecisionTreeClassifier(random_state=42)
dt_prunned.fit(X_train_scaled, y_train)

path = dt_prunned.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas  # list of candidate alphas

In [24]:
param_grid = {'ccp_alpha': ccp_alphas}
grid = GridSearchCV(DecisionTreeClassifier(random_state=0),
                    param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(X_train, y_train)

best_tree = grid.best_estimator_

In [25]:
best_dt_metrics = compute_metrics(
    data, best_tree, "Best Decision Tree Classifier", verbose=True
)

Best Decision Tree Classifier - train set metrics:
Accuracy: 0.8600, F1 Score: 0.6015, Precision: 0.9438, Recall: 0.4414
              precision    recall  f1-score   support

           0       0.85      0.99      0.92      3989
           1       0.94      0.44      0.60      1255

    accuracy                           0.86      5244
   macro avg       0.90      0.72      0.76      5244
weighted avg       0.87      0.86      0.84      5244

Best Decision Tree Classifier - val set metrics:
Accuracy: 0.8575, F1 Score: 0.5890, Precision: 0.9504, Recall: 0.4268
              precision    recall  f1-score   support

           0       0.85      0.99      0.91       998
           1       0.95      0.43      0.59       314

    accuracy                           0.86      1312
   macro avg       0.90      0.71      0.75      1312
weighted avg       0.87      0.86      0.84      1312

Best Decision Tree Classifier - test set metrics:
Accuracy: 0.8769, F1 Score: 0.7111, Precision: 0.8333, R

/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/n.protsiv/miniconda3/envs/loc-look-alike-venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


## Hyperparameter search for Random Forest

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 25, 50, 100],
    'max_depth': [None, 1, 3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2']
}

grid = GridSearchCV(RandomForestClassifier(random_state=0),
                    param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(X_train_scaled, y_train)

print("Best params:", grid.best_params_)
best_rf = grid.best_estimator_


Best params: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [27]:
best_rf_metrics = compute_metrics(
    data, best_rf, "Best Random Forest Classifier", verbose=True
)

Best Random Forest Classifier - train set metrics:
Accuracy: 0.9992, F1 Score: 0.9984, Precision: 1.0000, Recall: 0.9968
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3989
           1       1.00      1.00      1.00      1255

    accuracy                           1.00      5244
   macro avg       1.00      1.00      1.00      5244
weighted avg       1.00      1.00      1.00      5244

Best Random Forest Classifier - val set metrics:
Accuracy: 0.9337, F1 Score: 0.8643, Precision: 0.8471, Recall: 0.8822
              precision    recall  f1-score   support

           0       0.96      0.95      0.96       998
           1       0.85      0.88      0.86       314

    accuracy                           0.93      1312
   macro avg       0.90      0.92      0.91      1312
weighted avg       0.93      0.93      0.93      1312

Best Random Forest Classifier - test set metrics:
Accuracy: 0.8674, F1 Score: 0.7812, Precision: 0.6545, R

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score

# define your hyper-parameter grid
param_grid = {
    'n_estimators': [10, 25, 50, 100],
    'max_depth': [None, 2, 5, 8, 12],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt','log2']
}

# use stratified k-fold on *training* data only
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid, cv=cv,
    scoring='f1', n_jobs=-1,
)
grid.fit(X_train_scaled, y_train)

# now evaluate the *held-out* validation set
val_preds = grid.predict(X_val_scaled)
print("Val F1:", f1_score(y_val, val_preds))

test_preds = grid.predict(X_test_scaled)
print("Test F1:", f1_score(y_test, test_preds))

Val F1: 0.861198738170347
Test F1: 0.7848101265822784


In [29]:
best_rf_stratified = grid.best_estimator_

best_rf_stratified_metrics = compute_metrics(
    data, best_rf_stratified, "Best Random Forest Classifier (Stratified CV)", verbose=True
)

Best Random Forest Classifier (Stratified CV) - train set metrics:
Accuracy: 0.9992, F1 Score: 0.9984, Precision: 1.0000, Recall: 0.9968
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3989
           1       1.00      1.00      1.00      1255

    accuracy                           1.00      5244
   macro avg       1.00      1.00      1.00      5244
weighted avg       1.00      1.00      1.00      5244

Best Random Forest Classifier (Stratified CV) - val set metrics:
Accuracy: 0.9390, F1 Score: 0.8742, Precision: 0.8634, Recall: 0.8854
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       998
           1       0.86      0.89      0.87       314

    accuracy                           0.94      1312
   macro avg       0.91      0.92      0.92      1312
weighted avg       0.94      0.94      0.94      1312

Best Random Forest Classifier (Stratified CV) - test set metrics:
Accuracy